In [ ]:
from datetime import datetime

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.util as cutil
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import metpy.calc as mpcalc
import metpy.constants as mpconstants
from metpy.units import units
import numpy as np
from pyproj import Proj
from scipy.ndimage import gaussian_filter
import xarray as xr

In [ ]:
date = datetime(2025, 11, 21, 6)

# Current Data
# ds = xr.open_dataset('https://thredds.ucar.edu/thredds/dodsC/grib/NCEP/GFS/Global_0p5deg_ana/TP')

# UCAR RDA Archive - Global 1.0 deg Data from August 1, 1999 to current
# if datetime(1999, 7, 30, 18) <= date <= datetime(2007, 12, 6, 0):
#     ds = xr.open_dataset('https://tds.gdex.ucar.edu/thredds/dodsC/files/g/d083002/grib1/'
#                          f'{date:%Y}/{date:%Y.%m}/fnl_{date:%Y%m%d_%H}_00.grib1')
# elif date > datetime(2007, 12, 6, 0):
#     ds = xr.open_dataset('https://tds.gdex.ucar.edu/thredds/dodsC/files/g/d083002/grib2/'
#                          f'{date:%Y}/{date:%Y.%m}/fnl_{date:%Y%m%d_%H}_00.grib2')

# UCAR RDA Archive - Global 0.25 deg Data from 2015 to current
ds = xr.open_dataset('https://tds.gdex.ucar.edu/thredds/dodsC/files/g/d083003/'
                       f'{date:%Y}/{date:%Y%m}/gdas1.fnl0p25.{date:%Y%m%d%H}.f00.grib2')

# Local File
#ds = xr.open_dataset(f'groundhogs_day_blizzard/GFS_{date:%Y%m%d}_{date:%H}00.nc')

lat = ds.lat.sel(lat=slice(60, 10)).values
lon = ds.lon.sel(lon=slice(360-160, 360-50)).values

subset = dict(vertical=500 * units.hPa, time=date, lat=slice(60, 10), lon=slice(360-160, 360-50))
hght_500 = ds['Geopotential_height_isobaric'].metpy.sel(subset).metpy.quantify()
uwnd_500 = ds['u-component_of_wind_isobaric'].metpy.sel(subset).metpy.quantify()
vwnd_500 = ds['v-component_of_wind_isobaric'].metpy.sel(subset).metpy.quantify()
tmpk_500 = ds['Temperature_isobaric'].metpy.sel(subset)

hght_500s = mpcalc.smooth_n_point(hght_500, 9, 80)
uwnd_500s = mpcalc.smooth_n_point(uwnd_500, 9, 80)
vwnd_500s = mpcalc.smooth_n_point(vwnd_500, 9, 80)
tmpk_500s = mpcalc.smooth_n_point(tmpk_500, 9, 80)

subset['vertical'] = 700 * units.hPa
tmpk_700 = ds['Temperature_isobaric'].metpy.sel(subset).metpy.quantify()
uwnd_700 = ds['u-component_of_wind_isobaric'].metpy.sel(subset).metpy.quantify()
vwnd_700 = ds['v-component_of_wind_isobaric'].metpy.sel(subset).metpy.quantify()

tmpk_700s = mpcalc.smooth_n_point(tmpk_700, 9, 80)
uwnd_700s = mpcalc.smooth_n_point(uwnd_700, 9, 80)
vwnd_700s = mpcalc.smooth_n_point(vwnd_700, 9, 80)

subset['vertical'] = 300 * units.hPa
tmpk_300 = ds['Temperature_isobaric'].metpy.sel(subset).metpy.quantify()
uwnd_300 = ds['u-component_of_wind_isobaric'].metpy.sel(subset).metpy.quantify()
vwnd_300 = ds['v-component_of_wind_isobaric'].metpy.sel(subset).metpy.quantify()

tmpk_300s = mpcalc.smooth_n_point(tmpk_300, 9, 80)
uwnd_300s = mpcalc.smooth_n_point(uwnd_300, 9, 80)
vwnd_300s = mpcalc.smooth_n_point(vwnd_300, 9, 80)

In [ ]:
vtime = hght_500.time.data.astype('datetime64[ms]').astype('O')

mapcrs = ccrs.LambertConformal(central_longitude=-100, central_latitude=35, standard_parallels=(30, 60))
datacrs = ccrs.PlateCarree()

# Transform Coordinates ahead of time
lons, lats = np.meshgrid(lon, lat)
tlatlons = mapcrs.transform_points(ccrs.PlateCarree(), lons, lats)
clons = tlatlons[:,:,0]
clats = tlatlons[:,:,1]

# QGHT Forcing Terms
sigma = 2.0e-6 * units('m^2 Pa^-2 s^-2')
f0 = 1e-4 * units('s^-1')
Rd = mpconstants.Rd

avor_500 = mpcalc.absolute_vorticity(uwnd_500s, vwnd_500s)
vortadv_500 = mpcalc.advection(avor_500, uwnd_500s, vwnd_500s)

term_A = (f0 * vortadv_500.metpy.unit_array).to_base_units()

tadv_700 = mpcalc.advection(tmpk_700s, uwnd_700s, vwnd_700s)
tadv_300 = mpcalc.advection(tmpk_300s, uwnd_300s, vwnd_300s)

diff_tadv = ((Rd/(700 * units.hPa)*tadv_700.metpy.unit_array
              - Rd/(300 * units.hPa)*tadv_300.metpy.unit_array)/(400 * units.hPa)).to_base_units()

term_B = (-f0**2/sigma*diff_tadv).to_base_units()

# Q-Vectors and QVect Divergence
u_qvect, v_qvect = mpcalc.q_vector(uwnd_700s, vwnd_700s, tmpk_700s, 700*units.hPa)

Qdiv = mpcalc.divergence(u_qvect, v_qvect)

In [ ]:
clevs_700_tmpc = np.arange(-40, 41, 2)
clevs_500_hght = np.arange(0, 8000, 60)
clevs_QGHT = np.arange(-20, 20.5, 0.5)

# Use simple scheme for thinning wind barbs on plots
dlon = lon[1] - lon[0]
if dlon == 1:
    barb_spacing = 2
elif dlon == 0.5:
    barb_spacing = 5
elif dlon == 0.25:
    barb_spacing = 9
else:
    barb_spacing = 2
wind_slice = (slice(None, None, barb_spacing), slice(None, None, barb_spacing))

fig = plt.figure(1, figsize=(17,15))

ax3 = plt.subplot(111, projection=mapcrs)

ax3.set_extent([-130, -72, 20, 55], ccrs.PlateCarree())
ax3.add_feature(cfeature.COASTLINE.with_scale('50m'))
ax3.add_feature(cfeature.STATES.with_scale('50m'))

cf = ax3.contourf(clons, clats, -(term_A+term_B)*1e13, clevs_QGHT, cmap=plt.cm.bwr, extend='both')
plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, extendrect=True)

cs = ax3.contour(clons, clats, hght_500s, clevs_500_hght, colors='black')
plt.clabel(cs, fmt='%d')

cs2 = ax3.contour(clons, clats, -2*Qdiv*1e18, np.arange(-50,51,5), colors='grey')
plt.clabel(cs2, fmt='%d')

ax3.quiver(lons[wind_slice], lats[wind_slice],
           u_qvect[wind_slice].values, v_qvect[wind_slice].values,
           pivot='mid', scale=5e-11, scale_units='height', transform=ccrs.PlateCarree())

plt.title('GFS - 500-hPa Geo. HGHT (m), Total QGHT (Traditional) '
          '($*10^{13}$ s$^{-3}$) \n700-hPa Q-Vector/Divergence', loc='left')
plt.title(f'Valid Time: {vtime}', loc='right')

plt.savefig(f'GFS_QVector_Div_{date:%Y%m%d_%H}00.png', dpi=150, bbox_inches='tight')
plt.show()